The loop traverse network shown in figure below is a simple network with initial bearing of line P–Q fixed as 79° 37ʹ 22ʺ and coordinates of fixed-point P given as E = 1000m and N = 1150m. Table below indicates network distance and interior angle measurements made with a total station instrument having $SD_{HA}$ of ±3ʺ and $SD_{Distance}$ of 2mm + 2ppm. The $SD_{Angle}$ is ±4ʺ for a set of horizontal angular measurement at each station; the horizontal distance SDs are derived from distance specification with assumption that instrument and targets can be centered to an accuracy of ±0.8mm each. Adjust the measurements by transit method.

<div align="left">

|Observations|Figure|
|:-:|:-:|
|<img src="images/fig1.png" width="650">|<img src="images/fig2.png" width="300">|

</div>

In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, radians, degrees

def decimaltodms(decimal):
    dd = int(decimal)
    m = int((decimal - dd) * 60)
    s = (decimal - dd - m / 60) * 3600
    return f"{dd}° {m}' {s:.0f}\""

def calculate_bearing(bearing, angle):
    return np.where(bearing + 180 + angle > 360,
                    bearing + 180 + angle - 360,
                    bearing + 180 + angle)

def calculate_corrections(angles):
    sum_de = angles['de'].sum()
    sum_dn = angles['dn'].sum()
    abs_sum_de = abs(angles['de']).sum()
    abs_sum_dn = abs(angles['dn']).sum()

    angles['corr_de'] = -sum_de * (abs(angles['de']) / abs_sum_de)
    angles['corr_dn'] = -sum_dn * (abs(angles['dn']) / abs_sum_dn)

def calculate_adjusted_coordinates(angles, ep, np):
    angles['adj_e'] = 0.0
    angles['adj_n'] = 0.0

    angles.loc[0, 'adj_e'] = ep + angles.iloc[-1]['adj_de']
    angles.loc[0, 'adj_n'] = np + angles.iloc[-1]['adj_dn']

    for i in range(1, len(angles)):
        angles.loc[i, 'adj_e'] = angles.loc[i - 1, 'adj_e'] + angles.loc[i - 1, 'adj_de']
        angles.loc[i, 'adj_n'] = angles.loc[i - 1, 'adj_n'] + angles.loc[i - 1, 'adj_dn']

# Read the data from Excel
angles = pd.read_excel('observations01.xlsx')
angles['angle'] = angles['deg'] + angles['min'] / 60 + angles['sec'] / 3600
angles_sum = angles['angle'].sum()

# Calculate the correction per angle
correction = (angles_sum - (len(angles) - 2) * 180) / len(angles)

# Adjust the angles
angles['adjusted_angle'] = angles['angle'] - correction

# Calculate bearings
first_bearing = 79 + 37/60 + 22/3600
angles.loc[0, 'bearing'] = calculate_bearing(first_bearing, angles.loc[0, 'adjusted_angle'])

for i in range(1, len(angles)):
    angles.loc[i, 'bearing'] = calculate_bearing(angles.loc[i - 1, 'bearing'], angles.loc[i, 'adjusted_angle'])

# Calculate de and dn values
angles['de'] = angles['distance'] * np.sin(np.radians(angles['bearing']))
angles['dn'] = angles['distance'] * np.cos(np.radians(angles['bearing']))

# Calculate corrections
calculate_corrections(angles)

# Apply corrections
angles['adj_de'] = angles['de'] + angles['corr_de']
angles['adj_dn'] = angles['dn'] + angles['corr_dn']

# Calculate adjusted coordinates
ep, np = 1000.000, 1150.000
calculate_adjusted_coordinates(angles, ep, np)

# Display the results
display(angles[['from', 'at', 'to', 'angle', 'distance', 'bearing', 'de', 'dn',
                'corr_de', 'corr_dn', 'adj_de', 'adj_dn', 'adj_e', 'adj_n']].round(4))

sum_de = angles['de'].sum()
sum_dn = angles['dn'].sum()

linear_closure = (sum_de**2 + sum_dn**2)**0.5

print(f'Linear Misclosure: {linear_closure:.3f} m')

print(f'Ratio of Linear Closure to Perimeter: 1 : {(linear_closure / angles["distance"].sum())**-1:.5g}')

from math import atan2
bearing = degrees(atan2(sum_de, sum_dn))

# Adjust the bearing to be in the range [0, 360]
if bearing < 0:
    bearing += 360

print(f'Orientation of Linear Misclosure: {decimaltodms(bearing)}')

,from,at,to,angle,distance,bearing,de,dn,corr_de,corr_dn,adj_de,adj_dn,adj_e,adj_n
0,P,Q,R,119.7472,223.600,19.3693,74.1582,210.9443,0.0018,0.0009,74.1600,210.9452,1177.3335,1182.4733
1,Q,R,S,45.0028,237.184,244.3714,-213.8490,-102.5906,0.0051,0.0004,-213.8439,-102.5902,1251.4935,1393.4185
2,R,S,P,130.5972,145.775,194.9679,-37.6505,-140.8289,0.0009,0.0006,-37.6496,-140.8283,1037.6496,1290.8283
3,S,P,Q,64.6556,180.278,79.6228,177.3292,32.4731,0.0043,0.0001,177.3335,32.4733,1000.0000,1150.0000


Linear Misclosure: 0.012 m
Ratio of Linear Closure to Perimeter: 1 : 64170
Orientation of Linear Misclosure: 260° 11' 51"
